In [1]:
from glob import glob
import meshio
import numpy as np
from pyproj import Transformer

Allows us to convert between NZTM and NZMG

In [2]:
transform = Transformer.from_crs(2193, 27200, always_xy=True)

In [4]:
# Find remeshed STL files
stl_list = list(glob("fault_surfaces/*_remeshed.stl"))
# Loop through, calculating normals for each fault
for stl_name in stl_list:
    # File name for export (remove stl extension, replace with normals.txt)
    out_name = stl_name.split(".stl")[0] + "_normals.txt"
    # Read in mesh, identify triangles and vertices
    stl = meshio.read(stl_name)
    triangles = stl.cells_dict["triangle"]
    vertices = stl.points
    # combine triangle edges and vertices into single array
    triangles_xyz = vertices[triangles]
    # Calculate triangle centroids
    centroids = triangles_xyz.mean(axis=1)
    # convert centroids into format desires by Rafael
    nzmg_x, nzmg_y = transform.transform(centroids[:, 0], centroids[:, 1])
    # Reorder columns: N, E, down (down is positive)
    centroids_nzmg = np.column_stack([nzmg_y, nzmg_x, -1 * centroids[:, -1]])
    # Vectors between triangle vertices to create normals using cross product
    vec1 = np.array([tri[1] - tri[0] for tri in triangles_xyz])
    vec2 = np.array([tri[2] - tri[0] for tri in triangles_xyz])
    # Calculate normals
    crosses = np.cross(vec1, vec2)
    # Normals to unit vectors
    crosses /= np.linalg.norm(crosses, axis=1)[:, None]
    # Find normals that point downwards
    if np.mean(crosses[:, -1]) < 0:
        crosses *= -1
    # Reorder columns into Rafael's weird preferred format
    crosses_reordered = np.column_stack([crosses[:, 1], crosses[:, 0], -1 * crosses[:, -1]])
    # Combine centroids with unit normals
    normals = np.hstack([centroids_nzmg, crosses_reordered])
    # Write file
    np.savetxt(out_name, normals, delimiter=",", header="tri_cen_northing,tri_cen_easting,tri_cen_depth,normal_n,normal_e,normal_down",comments="", fmt="%.6f")
